# Laboratorio 04 - Procesamiento de Lenguaje Natural (NLP)

Integrantes:
- José Pablo Kiesling Lange, 21581
- Melissa Pérez Alarcón, 21385

### Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Embedding, concatenate

### 1. Importación de datos
- Utilice el conjunto de datos IMDB proporcionado por Keras. pero esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes, utilice las 50.000 palabras más frecuentes.

In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=50000)

print(f"Tamaño del conjunto de entrenamiento: {len(x_train)}")
print(f"Tamaño del conjunto de prueba: {len(x_test)}")

Tamaño del conjunto de entrenamiento: 25000
Tamaño del conjunto de prueba: 25000


### 2. Pre-procesamiento
- Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
- De las críticas, extraiga características (features) adicionales, por ejemplo. la longitud de la crítica, la proporción de palabras positivas/negativas y cualquier otra que considere pueda ser útil.

In [ ]:
# Longitud máxima de las secuencias
max_length = 500

# Relleno de las secuencias para que todas tengan la misma longitud
x_train_padded = pad_sequences(x_train, maxlen=max_length, padding='post')
x_test_padded = pad_sequences(x_test, maxlen=max_length, padding='post')

print(f"Tamaño de las secuencias de entrenamiento: {x_train_padded.shape}")
print(f"Tamaño de las secuencias de prueba: {x_test_padded.shape}")

In [ ]:
# Feature extraction
def calculate_word_ratios(text_sequence, positive_words, negative_words):
    pos_count = neg_count = total_count = 0
    for word_index in text_sequence:
        word = imdb.get_word_index().get(word_index, "")
        if word in positive_words:
            pos_count += 1
        if word in negative_words:
            neg_count += 1
        total_count += 1
    if total_count == 0:
        return (0, 0)
    return (pos_count / total_count, neg_count / total_count)

# Ejemplo de palabras positivas y negativas
positive_words = ['good', 'great', 'best', 'amazing', 'excellent', 'love']
negative_words = ['bad', 'worst', 'terrible', 'awful', 'horrible', 'hate']

# Aplicar la función a cada secuencia
train_ratios = [calculate_word_ratios(seq, positive_words, negative_words) for seq in x_train]
test_ratios = [calculate_word_ratios(seq, positive_words, negative_words) for seq in x_test]

# Convertir a numpy arrays
train_ratios_np = np.array(train_ratios)
test_ratios_np = np.array(test_ratios)

print(f"Características adicionales del conjunto de entrenamiento: {train_ratios_np.shape}")
print(f"Características adicionales del conjunto de prueba: {test_ratios_np.shape}")

### 3. Modelo
- Cree un modelo LSTM que acepte las características (features) adicionales junto con la secuencia de palabras.
- Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y tal vez alguna capa densamente conectada después de la LSTM. (ver también la referencia al final de este documento)

In [ ]:
# Definir la entrada para las secuencias de palabras
word_input = Input(shape=(max_length,), dtype='int32', name='word_input')

# Definir la entrada para las características adicionales
feature_input = Input(shape=(train_ratios_np.shape[1],), name='feature_input')

# Capa de embedding para las palabras
embedded_sequences = Embedding(input_dim=50000, output_dim=128, input_length=max_length)(word_input)

In [ ]:
# LSTM y Dropout
lstm_out = LSTM(64, return_sequences=True)(embedded_sequences)
lstm_out = Dropout(0.5)(lstm_out)
lstm_out = LSTM(64)(lstm_out)
lstm_out = Dropout(0.5)(lstm_out)

In [ ]:
# Combinar las salidas de las capas con las características adicionales
combined = concatenate([lstm_out, feature_input])

# Añadir capas densas
dense_out = Dense(64, activation='relu')(combined)
dense_out = Dropout(0.5)(dense_out)
output = Dense(1, activation='sigmoid')(dense_out)  # Salida binaria para análisis de sentimientos

In [ ]:
model = Model(inputs=[word_input, feature_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

### 4. Entrenamiento y Evaluación
- Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.